In [3]:
%load_ext autoreload
%autoreload 2

## Dataset Capture from Coppelia Scene

In [4]:
# if using .py: need to set environment variables->
# export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$COPPELIASIM_ROOT
# export QT_QPA_PLATFORM_PLUGIN_PATH=$COPPELIASIM_ROOT
# cd $COPPELIASIM_ROOT && ./coppeliaSim.sh

# if using .ipynb (Jupyter): need to config variables in kernel configs -> 
# https://stackoverflow.com/questions/73110604/vs-code-jupyter-integration-does-not-consider-custom-ld-library-path
# "env": {"LD_LIBRARY_PATH": "/home/dev/ws_coppelia/coppeliasim", "QT_QPA_PLATFORM_PLUGIN_PATH": "/home/dev/ws_coppelia/coppeliasim"}

import os, sys, time, pickle
LD_LIBRARY_PATH = 'LD_LIBRARY_PATH'
QT_QPA_PLATFORM_PLUGIN_PATH = 'QT_QPA_PLATFORM_PLUGIN_PATH'
print('LD_LIBRARY_PATH', LD_LIBRARY_PATH)
print('QT_QPA_PLATFORM_PLUGIN_PATH', QT_QPA_PLATFORM_PLUGIN_PATH)
assert LD_LIBRARY_PATH, "LD_LIBRARY_PATH needs to be set!"
assert QT_QPA_PLATFORM_PLUGIN_PATH, "QT_QPA_PLATFORM_PLUGIN_PATH needs to be set!"

from pyrep import PyRep
from pyrep.robots.arms.arm import Arm
from pyrep.objects import VisionSensor

PROJECT_NAME = 'dualarms_3cam'
SCENE = f'./scenes/{PROJECT_NAME}.ttt'
DATASET_SAVE_FILEPATH = f'./datasets/{PROJECT_NAME}/{PROJECT_NAME}.pkl'
SIMULATION_TIME = 2.0
HEADLESS    = True
IMAGE_SIZE  = 240
ARM_NAME = 'panda'
CAMERAS = ['cam1', 'cam2', 'cam3']


# setup and launch scene
pr = PyRep()
pr.launch(
    SCENE, 
    headless=HEADLESS,) 
pr.start() 


# wrapper for agents
class ArmWrapper(Arm):
    def __init__(
        self, 
        name: str,
        count: int = 0
    ):
        super().__init__(count, name, 7)  
        self.name = f'{name}{count}'

# setup agents
# from Pyrep's robots/arms/arm.py
# suffix = '' if count == 0 else '#%d' % (count - 1) 
agent1 = ArmWrapper(ARM_NAME,)
agent2 = ArmWrapper(ARM_NAME, count=1) 

agent_dict = dict(arms=[
    agent1, 
    agent2
])


class Observer():
    def __init__(
        self,
        CAMERA_NAMES:list,
        IMAGE_SIZE:int,
        AGENTS:dict,
    ):
        # setup cameras
        self.CAMERA_NAMES   = CAMERA_NAMES
        self.cameras    = dict()
        self.IMAGE_SIZE = IMAGE_SIZE

        # setup cameras and set resolution
        for cam_name in CAMERAS:
            cam = VisionSensor(cam_name)  
            cam.set_resolution([IMAGE_SIZE,IMAGE_SIZE])
            self.cameras[cam_name] = cam

        # list for storing observations
        self.observations = list()

        # setup agents
        self.agents = AGENTS

    def store_new_observation(self) -> None:
        obs = dict()
        
        # store cam obs
        for cam_name, cam in self.cameras.items():
            obs[cam_name] = self.get_camera_observation(cam) 

        # store arm joints obs
        for agent in self.agents['arms']:
            obs[agent.name] = self.get_agent_observation(agent)

        # store observation
        self.observations.append(obs)

    def get_agent_observation(self, agent) -> dict:
        joint_pos = agent.get_joint_positions()
        global_pos = agent.get_position()
        global_ang = agent.get_orientation()
        return dict( 
            joint_pos = joint_pos,
            global_pos = global_pos,
            global_ang = global_ang,
        )

    def get_camera_observation(self, cam:VisionSensor) -> dict:
        rgb = cam.capture_rgb()
        depth = cam.capture_depth()
        position = cam.get_position()
        resolution = cam.get_resolution()
        extrinsics = cam.get_matrix()
        intrinsics = cam.get_intrinsic_matrix()
        pointcloud = cam.capture_pointcloud()

        return dict(
            rgb=rgb,
            depth=depth,
            position=position,
            resolution=resolution,
            extrinsics=extrinsics,
            intrinsics=intrinsics,
            pointcloud=pointcloud)

    def save_observations(self, filepath:str) -> None:
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'wb') as f:
            pickle.dump(self.observations, f)


# setup observer
observer_handle = Observer(
    CAMERAS,
    IMAGE_SIZE,
    agent_dict,
)

print(f'\ndataset capture ({DATASET_SAVE_FILEPATH}) using PyRep started!')
# run simulation
start_time = time.time()
while (time.time() - start_time) < SIMULATION_TIME:

    # capture obs
    observer_handle.store_new_observation()

    # step simulation
    pr.step()

# save dataset
observer_handle.save_observations(DATASET_SAVE_FILEPATH)
print('dataset successfully output in: ', DATASET_SAVE_FILEPATH)

pr.stop()
pr.shutdown()

LD_LIBRARY_PATH LD_LIBRARY_PATH
QT_QPA_PLATFORM_PLUGIN_PATH QT_QPA_PLATFORM_PLUGIN_PATH

dataset capture (./datasets/dualarms_3cam/dualarms_3cam.pkl) using PyRep started!
dataset successfully output in:  ./datasets/dualarms_3cam/dualarms_3cam.pkl


## Perception Pipeline

In [5]:
# NOTES:
# "Vision Sensor" - In Coppelia scene (.ttt file) the "Vision sensor" is a generic sensor and a real world application would require appropriate calculations. 
# "Manual Offset for RBS URDF" - Manual offset for base of URDF required due to Coppelia Sim version 4.1.0. 
# Later versions don't require this offset but PyRep does not support those versions of coppelia Sim.
#  

In [6]:
# Root of Project
ROOT = '/home/dev/ws_percept/src'

# # Cameras
# IMAGE_SIZE  = 240
# CAMERAS = ['cam1', 'cam2', 'cam3']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = (-1.5, -1.5, -1.5, 1.50, 1.5, 1.5)

# Agents
AGENTS = [f'{ARM_NAME}0', f'{ARM_NAME}1']

SHOW_CPH_VIZ = False

ROBOT_URDF_PATH = 'outputs/testdata/franka_panda_cupoch/panda.urdf'
URDF_JOINT_PREFIX = 'panda_joint'

CUBIC_SIZE = 2.0
VOXEL_RESOLUTION = 30.0 # = 100.0
SPHERICAL_RADIUS = 0.03

import numpy as np
from copy import deepcopy
from scipy.spatial.transform import Rotation as R
import cupoch as cph


class PerceptionPipeline():
    def __init__(self):
        self.observations = None
        self.obs = None
        self.pcds = None
        self.pcd = None
        self.kin = None
        self.meshes_list = None
        self.voxels = None
        self.voxel_size = None
        self.v2s_positions = None
        self.spheres = None

    def load_dataset(self, DATASET_PATH) -> None:
        with open(DATASET_PATH, 'rb') as f:
            self.observations = pickle.load(f)

    def select_obs(self, n:int=1) -> None:
        self.obs = self.observations[n]
        self.get_pcds_for_registration()

    """
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        Get Pointclouds
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    """

    def constrain_scene_bounds(self, data) -> np.array:
        global SCENE_BOUNDS
        nX, nY, nZ, X, Y, Z = SCENE_BOUNDS
        X_mask = (data[:,0] > nX) & (data[:,0]<X)
        Y_mask = (data[:,1] > nY) & (data[:,1]<Y)
        Z_mask = (data[:,2] > nZ) & (data[:,2]<Z)
        mask = (X_mask)&(Y_mask)&(Z_mask) 
        return data[mask]

    def get_pcd_and_rgb(self, camera_obs:dict) -> tuple:
        pcd = camera_obs['pointcloud'].reshape(-2,3)
        rgb = camera_obs['rgb'].reshape(-2,3)
        return pcd, rgb
    
    def get_pcds_for_registration(self) -> None:
        global CAMERAS
        obs = self.obs
        assert obs is not None, "Did you select the obs?"

        pcds = list()
        for CAMERA in CAMERAS:
            camera_obs = obs[CAMERA]
            pcd, rgb = self.get_pcd_and_rgb(camera_obs)
            pcd = self.constrain_scene_bounds(pcd)
            pcds.append(pcd)

        # return pcds
        self.pcds = pcds

    def do_point_cloud_registration(
        self
    ) -> None: #cph.geometry.PointCloud:

        pcds = self.pcds
        assert pcds is not None, "Did you get the pcds?"
        assert len(pcds) > 1, "Are you using only 1 camera?"

        def register(
            source_gpu:cph.geometry.PointCloud, 
            target_gpu:cph.geometry.PointCloud,
        ) -> cph.geometry.PointCloud:

            threshold = 0.02 # what does this do?
            trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                                    [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

            # register pointclouds
            start = time.time()
            reg_p2p = cph.registration.registration_icp(
                source_gpu,
                target_gpu,
                threshold,
                trans_init.astype(np.float32),
                cph.registration.TransformationEstimationPointToPlane(),
            )

            source_gpu.transform(reg_p2p.transformation)

            # remove outliers
            NEIGHBOURS = 2
            source_gpu, ind = source_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)
            target_gpu, ind = target_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)


            elapsed_time = time.time() - start
            print("ICP (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms
        
            return source_gpu+target_gpu


        # load first and second pcd
        source_gpu = cph.geometry.PointCloud(
            cph.utility.HostVector3fVector(pcds[0])
        )
        target_gpu = cph.geometry.PointCloud(
            cph.utility.HostVector3fVector(pcds[1])
        )
        # register pcds
        source_gpu = register(source_gpu, target_gpu)

        # merge all other pcds with new source
        for pcd in pcds[2:]:
            target_gpu = cph.geometry.PointCloud(
                cph.utility.HostVector3fVector(pcd)
            )
            source_gpu = register(source_gpu, target_gpu)
        # return source_gpu
        self.pcd = source_gpu


    """
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        Do Robot Body Subtraction
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    """
    def get_tf_matrix(
        self,
        translation:np.array,
        rotation:np.array, 
    ) -> np.array:

        r = R.from_rotvec(rotation)
        r = r.as_matrix()

        tf = np.eye(4)
        tf[:3,:3] = r
        tf[:3,3] = translation

        return tf

    def manually_offset_base(   # used only for coppelia 4.1.0
        self,
        ref:np.array, 
        translation_offset
    ) -> np.array:
        offset = np.eye(4)
        offset[:3, 3] = translation_offset
        return np.dot(ref, offset)

    def get_mesh_using_forward_kinematics(
        self,
        joints_positions:np.array,
        global_position:np.array,
        global_rotation:np.array
    ):
        global URDF_JOINT_PREFIX, ROOT, ROBOT_URDF_PATH

        # load URDF model
        path = os.path.join(ROOT, ROBOT_URDF_PATH)
        self.kin = cph.kinematics.KinematicChain(path)

        # create joint_map, used for fk on urdf model
        joint_map = {'%s%d' % (URDF_JOINT_PREFIX, i+1): val for i, val \
                    in enumerate(joints_positions)} 

        # create transformation matrix
        tf_matrix = self.get_tf_matrix(global_position, global_rotation)

        # manual offset, to be removed 
        offset = np.array([0.04, 0.0, -0.068])
        tf_matrix = self.manually_offset_base(tf_matrix, offset)
        
        # do forward kinematics
        poses = self.kin.forward_kinematics(joint_map, tf_matrix)
        
        # store mesh geometries
        meshes = self.kin.get_transformed_visual_geometry_map(poses)
        
        return list(meshes.values())

    def get_bb_from_mesh(
        self,
        mesh:cph.geometry.TriangleMesh
    ) -> cph.geometry.AxisAlignedBoundingBox:

        AABB = mesh.get_axis_aligned_bounding_box()
        if AABB.is_empty():
            print('empty mesh detected, skipped')
            return None       
        else:
            return AABB


    def perform_rbs_on_pointcloud_using_bb(
        self, 
    )->tuple:
        obs = self.obs
        pcd = self.pcd

        assert obs is not None, "Did you get select the obs?"
        assert pcd is not None, "Did you perform PCD registration?"

        n = len(pcd.points)
        masks = list()
        meshes_list = list()
        for agent in AGENTS:
            print('Agent: %s' % agent)
            agent_obs = obs[agent]
            
            # get all meshes of the agent using fk
            meshes = self.get_mesh_using_forward_kinematics(
                agent_obs['joint_pos'],
                agent_obs['global_pos'],
                agent_obs['global_ang'],
            )
            meshes_list += meshes

            # find points within BBs and create masks
            start = time.time()
            for mesh in meshes:
                bb = self.get_bb_from_mesh(mesh)

                if bb is not None:
                    mask = np.zeros(n, dtype=bool)

                    indices_points_within_bb = np.asarray(
                        bb.get_point_indices_within_bounding_box(
                            pcd.points
                        ).cpu()
                    )
                    if len(indices_points_within_bb)>0:
                        mask[indices_points_within_bb] = True
                        masks.append(mask)

            elapsed_time = time.time() - start
            print("RBS (CPU+GPU) [sec]:", elapsed_time)


        mask = np.column_stack(tuple(masks)).any(axis=1)
        mask = ~mask # inverting mask to get pcd which were not within any BBs
        
        pcd_arr = np.asarray(pcd.points.cpu())
        NEIGHBOURS = 10
        pcd = cph.geometry.PointCloud(cph.utility.HostVector3fVector(pcd_arr[mask]))
        pcd = pcd.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=1.0)
        # return  pcd[0], meshes_list
        self.pcd = pcd[0] 
        self.meshes_list = meshes_list

    """
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        Voxelization
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    """
    def convert_pointcloud_to_voxels(self):
        global CUBIC_SIZE, VOXEL_RESOLUTION
        cubic_size = CUBIC_SIZE
        voxel_resolution = VOXEL_RESOLUTION
        voxel_size = cubic_size / voxel_resolution

        pcd = self.pcd
        assert pcd is not None, "Did you do Point Cloud Registration?"

        # create voxel grid
        start = time.time()
        voxels = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
            pcd,
            voxel_size=cubic_size / voxel_resolution,
            min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
            max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2),
        )
        elapsed_time = time.time() - start
        print("Voxelization (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

        # return voxels, voxel_size
        self.voxels = voxels
        self.voxel_size = voxel_size

    """
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        V2S - Voxel To Sphere
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    """
    def voxel2sphere(self) -> None:
        voxel_size = self.voxel_size
        voxels = self.voxels
        
        assert voxel_size is not None, "Did you not convert pcd to voxels?"
        assert voxels is not None, "Did you not convert pcd to voxels?"

        start = time.time()
        v = voxels.voxels.cpu()
        offset = voxels.get_min_bound()
        positions = np.array(list(v.keys()))
        positions = (positions - np.array([
                        min(positions[:,0]), 
                        min(positions[:,1]), 
                        min(positions[:,2])]))
        positions = positions*voxel_size
        positions += (offset + voxel_size/2)

        elapsed_time = time.time() - start
        print("Voxel2Sphere (CPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

        self.v2s_positions = positions

    def get_sphere_viz(self, meshcolor:bool=True):
        global SPHERICAL_RADIUS
        voxels = self.voxels
        voxel_size = self.voxel_size
        radius = SPHERICAL_RADIUS
        resolution = 5
        positions = self.v2s_positions

        assert positions is not None, "Did you convert Voxels to Spheres (V2S)?"

        spheres = cph.geometry.TriangleMesh.create_sphere(
            radius=radius, 
            resolution=resolution).translate(positions[0])
        for pos in positions[1:]:
            spheres += cph.geometry.TriangleMesh.create_sphere(
                radius=radius, 
                resolution=resolution).translate(pos)

        if meshcolor:
            import matplotlib.pyplot as plt
            vertices = np.asarray(spheres.vertices.cpu())
            z_values = vertices[:, 2]

            # Normalize the z values to range [0, 1]
            z_min = z_values.min()
            z_max = z_values.max()
            z_norm = (z_values - z_min) / (z_max - z_min)

            # Create a color map (using a matplotlib colormap, e.g., 'viridis')
            cmap = plt.get_cmap('turbo')
            colors = cmap(z_norm)[:, :3]  # Get RGB values

            # Apply the colors to the mesh
            spheres.vertex_colors = cph.utility.Vector3fVector(colors)
        self.spheres = spheres

    def run(self):
        global DATASET_SAVE_FILEPATH
        self.load_dataset(DATASET_SAVE_FILEPATH)
        self.select_obs(1)
        self.do_point_cloud_registration()
        self.perform_rbs_on_pointcloud_using_bb()
        self.convert_pointcloud_to_voxels()
        self.voxel2sphere()


pipe = PerceptionPipeline()
pipe.run()


ICP (GPU) [sec]: 0.037661075592041016
ICP (GPU) [sec]: 0.04591488838195801
Agent: panda0
empty mesh detected, skipped
empty mesh detected, skipped
RBS (CPU+GPU) [sec]: 0.0236051082611084
Agent: panda1
empty mesh detected, skipped
empty mesh detected, skipped
RBS (CPU+GPU) [sec]: 0.021513938903808594
Voxelization (GPU) [sec]: 0.003942012786865234
Voxel2Sphere (CPU) [sec]: 0.014079093933105469


In [7]:
pipe.get_sphere_viz()
cph.visualization.draw_geometries([pipe.spheres])

## YAML Output

In [8]:
SPHERICAL_RADIUS

0.03

In [16]:
import yaml

# YAML_FILE_BASENAME = 'dual_arms_static3'
# YAML_OUTPUT_FILE = f'./datasets/{PROJECT_NAME}/{YAML_FILE_BASENAME}.yaml'

YAML_OUTPUT_FILE = '/home/dev/catkin_ws/pmaf_ws/src/bimanual_planning_ros/config/tasks/dual_arms_static3.yaml'

class YAMLObstacles():
    def __init__(self):
        self.vel = [0.0, 0.0, 0.0]
        self.isep = '  '
        self.osep = '      '

    def dump_item(self, pos:np.array, radius:float):

        string = f"{self.osep}- pos: {(pos+np.array([0.40, -0.1,0.0])).tolist()}"
        string += f"\n{self.osep}{self.isep}radius: {radius}"
        string += f"\n{self.osep}{self.isep}vel: {self.vel}"
        return string
    
    def output_to_yaml_file(
        self, 
        obstacle_positions:np.array,
        radius:float,
        read_until:int= 1742
    ) -> None:
        global YAML_OUTPUT_FILE
        yaml_str = "      obstacles:\n"
        for pos in obstacle_positions:
            yaml_str += self.dump_item(pos, radius)
            yaml_str += '\n'
        # print(yaml_str)

        contents = None
        with open(YAML_OUTPUT_FILE, 'r') as file:
            contents = file.read()

        with open(YAML_OUTPUT_FILE, 'w+') as f:
            f.write(contents[:read_until]+"\n"+yaml_str)



yaml_obstacles = YAMLObstacles()
yaml_obstacles.output_to_yaml_file(pipe.v2s_positions, SPHERICAL_RADIUS*1.5)


In [ ]:
# goal_pos: [0.05, 0.0, 0.45]